torch.linalg.lstsq

Diese Test findest du im KAN Repo im Ordner kan-> splines.py

In [30]:
import torch

# Beispielhafte Initialisierung von mat und y_eval
mat = torch.randn(64, 1, 10000, 6)
y_eval = torch.randn(64, 1, 10000, 1)
device = 'cpu'  # oder 'cuda', wenn eine GPU verwendet wird

# Lösen des linearen Gleichungssystems mit torch.linalg.lstsq
result = torch.linalg.lstsq(mat.to(device), y_eval.to(device), driver='gelsy' if device == 'cpu' else 'gels')

# Extrahieren der Lösung
coef = result.solution[:,:,:,0]

# Überprüfen der Dimensionen der Lösung
print(f"coef shape: {coef.shape}")

coef shape: torch.Size([64, 1, 6])


In [31]:
import torch

# Einlesen der Tensoren
x_eval = torch.load('./x_eval.pt')
grid = torch.load('./grid.pt')
y_eval_tensor = torch.load('./y_eval_tensor.pt')
mat_per_tensor = torch.load('./mat_per_tensor.pt')

In [32]:
def check_tensor(tensor, tensor_name="tensor"):
    print(f"Checking tensor: {tensor_name}")
    
    # Überprüfen der Form des gesamten Tensors
    print(f"Shape of {tensor_name}: {tensor.shape}")
    
    # Überprüfen, ob der Tensor NaN-Werte enthält
    contains_nan = torch.isnan(tensor).any()
    print(f"{tensor_name} contains NaN: {contains_nan}")
    
    # Überprüfen, ob der Tensor nur aus NaN-Werten besteht
    only_nan = torch.isnan(tensor).all()
    print(f"{tensor_name} contains only NaN: {only_nan}")

    # Überprüfen, ob der Tensor nur aus Nullen besteht
    contains_only_zeros = torch.eq(tensor, 0).all()
    print(f"{tensor_name} contains only zeros: {contains_only_zeros}")
    
    # Höchsten und niedrigsten Wert berechnen, NaNs ignorieren
    if contains_nan:
        tensor_without_nan = tensor[~torch.isnan(tensor)]
        if tensor_without_nan.numel() > 0:
            max_value = torch.max(tensor_without_nan)
            min_value = torch.min(tensor_without_nan)
            print(f"Max value in {tensor_name} (ignoring NaNs): {max_value.item()}")
            print(f"Min value in {tensor_name} (ignoring NaNs): {min_value.item()}")
        else:
            print(f"Cannot compute max/min values in {tensor_name}: Tensor contains only NaNs")
    else:
        max_value = torch.max(tensor)
        min_value = torch.min(tensor)
        print(f"Max value in {tensor_name}: {max_value.item()}")
        print(f"Min value in {tensor_name}: {min_value.item()}")
    
    print()

In [39]:
# Überprüfen der Tensoren
check_tensor(x_eval, "x_eval")
check_tensor(grid, "grid")
check_tensor(y_eval_tensor, "y_eval_tensor")
check_tensor(mat_per_tensor, "mat_per_tensor")
check_tensor(mat_self, "mat_self")

Checking tensor: x_eval
Shape of x_eval: torch.Size([10000, 784])
x_eval contains NaN: False
x_eval contains only NaN: False
x_eval contains only zeros: False
Max value in x_eval: 255.0
Min value in x_eval: 0.0

Checking tensor: mat_tensor
Shape of mat_tensor: torch.Size([10000, 784, 6])
mat_tensor contains NaN: True
mat_tensor contains only NaN: False
mat_tensor contains only zeros: False
Max value in mat_tensor (ignoring NaNs): 0.6666666865348816
Min value in mat_tensor (ignoring NaNs): 0.0

Checking tensor: grid
Shape of grid: torch.Size([784, 10])
grid contains NaN: False
grid contains only NaN: False
grid contains only zeros: False
Max value in grid: 510.0
Min value in grid: -255.0

Checking tensor: y_eval_tensor
Shape of y_eval_tensor: torch.Size([10000, 784, 1])
y_eval_tensor contains NaN: False
y_eval_tensor contains only NaN: False
y_eval_tensor contains only zeros: False
Max value in y_eval_tensor: 0.1982683539390564
Min value in y_eval_tensor: -0.19842682778835297

Checking 

Test_B_batch

In [46]:
def B_batch(x, grid, k=0, extend=True, device='cpu'):
    '''
    evaludate x on B-spline bases
    
    Args:
    -----
        x : 2D torch.tensor
            inputs, shape (number of splines, number of samples)
        grid : 2D torch.tensor
            grids, shape (number of splines, number of grid points)
        k : int
            the piecewise polynomial order of splines.
        extend : bool
            If True, k points are extended on both ends. If False, no extension (zero boundary condition). Default: True
        device : str
            devicde
    
    Returns:
    --------
        spline values : 3D torch.tensor
            shape (number of splines, number of B-spline bases (coeffcients), number of samples). The numbef of B-spline bases = number of grid points + k - 1.
      
    Example
    -------
    >>> num_spline = 5
    >>> num_sample = 100
    >>> num_grid_interval = 10
    >>> k = 3
    >>> x = torch.normal(0,1,size=(num_spline, num_sample))
    >>> grids = torch.einsum('i,j->ij', torch.ones(num_spline,), torch.linspace(-1,1,steps=num_grid_interval+1))
    >>> B_batch(x, grids, k=k).shape
    torch.Size([5, 13, 100])
    '''

    '''# x shape: (size, x); grid shape: (size, grid)
    def extend_grid(grid, k_extend=0):
        # pad k to left and right
        # grid shape: (batch, grid)
        h = (grid[:, [-1]] - grid[:, [0]]) / (grid.shape[1] - 1)

        for i in range(k_extend):
            grid = torch.cat([grid[:, [0]] - h, grid], dim=1)
            grid = torch.cat([grid, grid[:, [-1]] + h], dim=1)
        grid = grid.to(device)
        return grid

    if extend == True:
        grid = extend_grid(grid, k_extend=k)

    grid = grid.unsqueeze(dim=2).to(device)
    x = x.unsqueeze(dim=1).to(device)

    if k == 0:
        value = (x >= grid[:, :-1]) * (x < grid[:, 1:])
    else:
        B_km1 = B_batch(x[:, 0], grid=grid[:, :, 0], k=k - 1, extend=False, device=device)
        value = (x - grid[:, :-(k + 1)]) / (grid[:, k:-1] - grid[:, :-(k + 1)]) * B_km1[:, :-1] + (
                    grid[:, k + 1:] - x) / (grid[:, k + 1:] - grid[:, 1:(-k)]) * B_km1[:, 1:]'''
    
    x = x.unsqueeze(dim=2).to(device)
    grid = grid.unsqueeze(dim=0).to(device)
    
    if k == 0:
        value = (x >= grid[:, :, :-1]) * (x < grid[:, :, 1:])
    else:
        B_km1 = B_batch(x[:,:,0], grid=grid[0], k=k - 1, device=device)
        epsilon = 1e-10
        trick1 = (grid[:, :, k:-1] - grid[:, :, :-(k + 1)])+epsilon
        trick2 = (grid[:, :, k + 1:] - grid[:, :, 1:(-k)])+epsilon
        value = (x - grid[:, :, :-(k + 1)]) / trick1 * B_km1[:, :, :-1] + (
                    grid[:, :, k + 1:] - x) / trick2 * B_km1[:, :, 1:]
    return value

Orginal B_batch

In [34]:
def B_batch(x, grid, k=0, extend=True, device='cpu'):
    '''
    evaludate x on B-spline bases
    
    Args:
    -----
        x : 2D torch.tensor
            inputs, shape (number of splines, number of samples)
        grid : 2D torch.tensor
            grids, shape (number of splines, number of grid points)
        k : int
            the piecewise polynomial order of splines.
        extend : bool
            If True, k points are extended on both ends. If False, no extension (zero boundary condition). Default: True
        device : str
            devicde
    
    Returns:
    --------
        spline values : 3D torch.tensor
            shape (number of splines, number of B-spline bases (coeffcients), number of samples). The numbef of B-spline bases = number of grid points + k - 1.
      
    Example
    -------
    >>> num_spline = 5
    >>> num_sample = 100
    >>> num_grid_interval = 10
    >>> k = 3
    >>> x = torch.normal(0,1,size=(num_spline, num_sample))
    >>> grids = torch.einsum('i,j->ij', torch.ones(num_spline,), torch.linspace(-1,1,steps=num_grid_interval+1))
    >>> B_batch(x, grids, k=k).shape
    torch.Size([5, 13, 100])
    '''

    '''# x shape: (size, x); grid shape: (size, grid)
    def extend_grid(grid, k_extend=0):
        # pad k to left and right
        # grid shape: (batch, grid)
        h = (grid[:, [-1]] - grid[:, [0]]) / (grid.shape[1] - 1)

        for i in range(k_extend):
            grid = torch.cat([grid[:, [0]] - h, grid], dim=1)
            grid = torch.cat([grid, grid[:, [-1]] + h], dim=1)
        grid = grid.to(device)
        return grid

    if extend == True:
        grid = extend_grid(grid, k_extend=k)

    grid = grid.unsqueeze(dim=2).to(device)
    x = x.unsqueeze(dim=1).to(device)

    if k == 0:
        value = (x >= grid[:, :-1]) * (x < grid[:, 1:])
    else:
        B_km1 = B_batch(x[:, 0], grid=grid[:, :, 0], k=k - 1, extend=False, device=device)
        value = (x - grid[:, :-(k + 1)]) / (grid[:, k:-1] - grid[:, :-(k + 1)]) * B_km1[:, :-1] + (
                    grid[:, k + 1:] - x) / (grid[:, k + 1:] - grid[:, 1:(-k)]) * B_km1[:, 1:]'''
    
    x = x.unsqueeze(dim=2).to(device)
    grid = grid.unsqueeze(dim=0).to(device)
    
    if k == 0:
        value = (x >= grid[:, :, :-1]) * (x < grid[:, :, 1:])
    else:
        B_km1 = B_batch(x[:,:,0], grid=grid[0], k=k - 1, device=device)
        
        value = (x - grid[:, :, :-(k + 1)]) / (grid[:, :, k:-1] - grid[:, :, :-(k + 1)]) * B_km1[:, :, :-1] + (
                    grid[:, :, k + 1:] - x) / (grid[:, :, k + 1:] - grid[:, :, 1:(-k)]) * B_km1[:, :, 1:]
    return value

In [40]:
grid.shape

torch.Size([784, 10])

In [41]:
grid

Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [35]:
x_eval.shape

torch.Size([10000, 784])

In [36]:
x_eval

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [47]:
mat_self = B_batch(x_eval, grid, 3)

In [48]:
mat_self

tensor([[[0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.]],

        ...,

        [[0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0.